In [1]:
## Estimación de viviendas para gobierno federal de Claudia Sheinbaum
##Borrar datos del entorno
rm(list=ls())

#Directorio para descargar de datos
#Esto se debe de cambiar en cada computadora
#setwd("D:/datos")
#creamos carpeta en donde almacenaremos los archivos
dir.create("envi", showWarnings=F)

In [2]:
# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr,showtext)
#Fuente para los gráficos
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman



In [3]:
#Descarga de archivos====
url<-"https://www.inegi.org.mx/contenidos/programas/envi/2020/microdatos/envi_2020_base_de_datos_csv.zip"

temp <- tempfile()
download.file(url,
              mode = "wb",
              destfile = temp)
unzip(temp, exdir = "envi")
unlink(temp)

In [20]:
demog <- read.csv("envi/Bases de datos/TSDEM.csv") %>%
  janitor::clean_names() %>%
  # Clasificar población ocupada
  mutate(po = case_when(
    p3_1 %in% c(1, 2) ~ "ocupada",
    !(p3_1 %in% c(1, 2)) & p3_2 %in% c(1, 2, 3) ~ "ocupada",
    TRUE ~ "otros"
  ))

head(demog)

,folio,viv_sel,hogar,n_ren,paren,sexo,edad,p2_5,p2_6,niv,⋯,p3_3,p3_4,p3_4a,p3_5,ent,tloc,est_dis,upm_dis,factor,po
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,1,1,1,1,1,1,61,2,1,3,⋯,4,99888,NA,4,1,1,2,1,206,ocupada
2,1,1,1,4,7,2,64,2,2,3,⋯,NA,NA,NA,NA,1,1,2,1,206,otros
3,1,1,1,2,7,2,67,2,1,2,⋯,4,3500,3,4,1,1,2,1,206,ocupada
4,1,2,1,1,1,1,74,2,2,2,⋯,4,3000,1,4,1,1,2,1,206,ocupada
5,1,2,1,2,2,2,72,2,2,2,⋯,NA,NA,NA,NA,1,1,2,1,206,otros
6,1,2,1,3,3,1,32,2,2,8,⋯,2,1500,1,1,1,1,2,1,206,ocupada


In [21]:
str(demog)

'data.frame':	202595 obs. of  24 variables:
 $ folio  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ viv_sel: int  1 1 1 2 2 2 3 4 4 4 ...
 $ hogar  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ n_ren  : int  1 4 2 1 2 3 1 1 2 3 ...
 $ paren  : int  1 7 7 1 2 3 1 1 2 3 ...
 $ sexo   : int  1 2 2 1 2 1 2 2 1 2 ...
 $ edad   : int  61 64 67 74 72 32 57 55 74 23 ...
 $ p2_5   : int  2 2 2 2 2 2 2 2 2 2 ...
 $ p2_6   : int  1 2 1 2 2 2 1 2 2 2 ...
 $ niv    : int  3 3 2 2 2 8 8 4 3 8 ...
 $ gra    : int  3 3 6 6 6 2 5 3 3 2 ...
 $ p2_8   : int  6 5 6 5 5 6 3 5 5 6 ...
 $ p3_1   : int  2 4 2 1 6 1 1 6 4 5 ...
 $ p3_2   : int  NA 4 NA NA 4 NA NA 4 4 4 ...
 $ p3_3   : int  4 NA 4 4 NA 2 4 NA NA NA ...
 $ p3_4   : int  99888 NA 3500 3000 NA 1500 99888 NA NA NA ...
 $ p3_4a  : int  NA NA 3 1 NA 1 NA NA NA NA ...
 $ p3_5   : int  4 NA 4 4 NA 1 4 NA NA NA ...
 $ ent    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ tloc   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ est_dis: int  2 2 2 2 2 2 2 2 2 2 ...
 $ upm_dis: int  1 1 1 1 1 1 1 1 1 1 ...


In [22]:
options(survey.lonely.psu="certainty")

dm<-demog%>%
as_survey_design(
id=upm_dis,
strata=est_dis,
weights=factor)

In [ ]:
#Población ocupada total
print("Población ocupada total en CDMX")
dm%>%
filter(ent==9)%>%
#Filtro de edad
filter(edad >= 15 & edad <= 98)%>%
#Filtrar ocupadas
filter(po=="ocupada")%>%
summarise(pob=survey_total(vartype="cv"))

[1] "Población ocupada total en CDMX"


pob,pob_cv
<dbl>,<dbl>
4493232,0.02524955


In [38]:
#Población ocupada por acceso a financiamiento
print("Población ocupada por acceso a financiamiento en CDMX")
dm%>%
filter(ent==9)%>%
#Filtro de edad
filter(edad >= 15 & edad <= 98)%>%
filter(po=="ocupada")%>%
#Agrupar por acceso a financiamiento
group_by(p3_5)%>%

summarise(pob=survey_total(vartype="cv"),
pct=survey_prop(vartype = "cv"))%>%
mutate(pct = round(pct * 100,1))

[1] "Población ocupada por acceso a financiamiento en CDMX"


p3_5,pob,pob_cv,pct,pct_cv
<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1310763,0.04609662,29.2,0.03839378
2,372112,0.07796306,8.3,0.07618376
3,27671,0.28183921,0.6,0.27904723
4,2599716,0.03285925,57.9,0.02111515
9,112071,0.15804322,2.5,0.15454070
NA,70899,0.17879863,1.6,0.17498943
